In [7]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)# 讀取圖片數據集
sess = tf.InteractiveSession()# 創建session

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [9]:
# 一，函數聲明部分

def weight_variable(shape):
	# 正態分布，標準差為0.1，默認最大為1，最小為-1，均值為0
		initial = tf.truncated_normal(shape, stddev=0.1)
		return tf.Variable(initial)
def bias_variable(shape):
	# 創建一個結構為shape矩陣也可以說是數組shape聲明其行列，初始化所有值為0.1
		initial = tf.constant(0.1, shape=shape)
		return tf.Variable(initial)
def conv2d(x, W):  
	# 卷積遍歷各方向步數為1，SAME：邊緣外自動補0，遍歷相乘
	return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')  
def max_pool_2x2(x):  
	# 池化卷積結果（conv2d）池化層采用kernel大小為2*2，步數也為2，周圍補0，取最大值。數據量縮小了4倍
	return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')  


In [10]:
# 二，定義輸入輸出結構

# 聲明一個占位符，None表示輸入圖片的數量不定，28*28圖片分辨率
xs = tf.placeholder(tf.float32, [None, 28*28]) 
# 類別是0-9總共10個類別，對應輸出分類結果
ys = tf.placeholder(tf.float32, [None, 10]) 
keep_prob = tf.placeholder(tf.float32)
# x_image又把xs reshape成了28*28*1的形狀，因為是灰色圖片，所以通道是1.作為訓練時的input，-1代表圖片數量不定
x_image = tf.reshape(xs, [-1, 28, 28, 1]) 

In [11]:
# 三，搭建網絡,定義算法公式，也就是forward時的計算

## 第一層卷積操作 ##
# 第一二參數值得卷積核尺寸大小，即patch，第三個參數是圖像通道數，第四個參數是卷積核的數目，代表會出現多少個卷積特征圖像;
W_conv1 = weight_variable([5, 5, 1, 32]) 
# 對於每一個卷積核都有一個對應的偏置量。
b_conv1 = bias_variable([32])  
# 圖片乘以卷積核，並加上偏執量，卷積結果28x28x32
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)  
# 池化結果14x14x32 卷積結果乘以池化卷積核
h_pool1 = max_pool_2x2(h_conv1) 

In [12]:
## 第二層卷積操作 ##   
# 32通道卷積，卷積出64個特征  
w_conv2 = weight_variable([5,5,32,64]) 
# 64個偏執數據
b_conv2  = bias_variable([64]) 
# 註意h_pool1是上一層的池化結果，#卷積結果14x14x64
h_conv2 = tf.nn.relu(conv2d(h_pool1,w_conv2)+b_conv2)  
# 池化結果7x7x64
h_pool2 = max_pool_2x2(h_conv2)  
# 原圖像尺寸28*28，第一輪圖像縮小為14*14，共有32張，第二輪後圖像縮小為7*7，共有64張

In [13]:
## 第三層全連接操作 ##
# 二維張量，第一個參數7*7*64的patch，也可以認為是只有一行7*7*64個數據的卷積，第二個參數代表卷積個數共1024個
W_fc1 = weight_variable([7*7*64, 1024]) 
# 1024個偏執數據
b_fc1 = bias_variable([1024]) 
# 將第二層卷積池化結果reshape成只有一行7*7*64個數據# [n_samples, 7, 7, 64] ->> [n_samples, 7*7*64]
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64]) 
# 卷積操作，結果是1*1*1024，單行乘以單列等於1*1矩陣，matmul實現最基本的矩陣相乘，不同於tf.nn.conv2d的遍歷相乘，自動認為是前行向量後列向量
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1) 

In [16]:
## 第三層全連接操作 ##
# 二維張量，第一個參數7*7*64的patch，也可以認為是只有一行7*7*64個數據的卷積，第二個參數代表卷積個數共1024個
W_fc1 = weight_variable([7*7*64, 1024]) 
# 1024個偏執數據
b_fc1 = bias_variable([1024]) 
# 將第二層卷積池化結果reshape成只有一行7*7*64個數據# [n_samples, 7, 7, 64] ->> [n_samples, 7*7*64]
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64]) 
# 卷積操作，結果是1*1*1024，單行乘以單列等於1*1矩陣，matmul實現最基本的矩陣相乘，不同於tf.nn.conv2d的遍歷相乘，自動認為是前行向量後列向量
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1) 

# dropout操作，減少過擬合，其實就是降低上一層某些輸入的權重scale，甚至置為0，升高某些輸入的權值，甚至置為2，防止評測曲線出現震蕩，個人覺得樣本較少時很必要
# 使用占位符，由dropout自動確定scale，也可以自定義，比如0.5，根據tensorflow文檔可知，程序中真實使用的值為1/0.5=2，也就是某些輸入乘以2，同時某些輸入乘以0
keep_prob = tf.placeholder(tf.float32) 
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob) #對卷積結果執行dropout操作

In [17]:
## 第四層輸出操作 ##
# 二維張量，1*1024矩陣卷積，共10個卷積，對應我們開始的ys長度為10
W_fc2 = weight_variable([1024, 10])  
b_fc2 = bias_variable([10])  
# 最後的分類，結果為1*1*10 softmax和sigmoid都是基於logistic分類算法，一個是多分類一個是二分類
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2) 

In [22]:
# 四，定義loss(最小誤差概率)，選定優化優化loss，
cross_entropy = -tf.reduce_sum(ys * tf.log(y_conv)) # 定義交叉熵為loss函數  
train_step = tf.train.AdamOptimizer(0.5).minimize(cross_entropy) # 調用優化器優化，其實就是通過餵數據爭取cross_entropy最小化  

In [23]:
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(ys,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.global_variables_initializer().run()

In [60]:
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], ys: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], ys: batch[1], keep_prob: 0.5})    
print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, ys: mnist.test.labels, keep_prob: 1.0}))

NameError: name 'tt' is not defined

In [71]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
      train_accuracy = accuracy.eval(feed_dict={
          x: batch[0], y_: batch[1], keep_prob: 1.0})
      print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

  print('test accuracy %g' % accuracy.eval(feed_dict={
      x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

NameError: name 'x' is not defined